# Colab Setup: SSH via Cloudflare + GPU Monitoring

1. Run Cell 1 to start SSH access via Cloudflare tunnel
2. Run Cell 2 to start GPU usage logging every 1 minute

In [ ]:
# Cell 1: SSH via Cloudflare Tunnel

import subprocess, time, re

# Set root password
password = "persona123"
subprocess.run(f"echo 'root:{password}' | chpasswd", shell=True)

# Download and install cloudflared
subprocess.run("wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb", shell=True)
subprocess.run("dpkg -i cloudflared-linux-amd64.deb", shell=True)

# Start Cloudflare tunnel in background, log to file
subprocess.run("pkill cloudflared", shell=True)
time.sleep(1)
subprocess.Popen(
    "cloudflared tunnel --url ssh://localhost:22 --no-autoupdate 2>&1 | tee /tmp/cloudflared.log",
    shell=True,
)

# Wait and extract tunnel URL
print("Starting Cloudflare tunnel...")
time.sleep(15)

with open("/tmp/cloudflared.log", "r") as f:
    log = f.read()

match = re.search(r'https://([\w-]+\.trycloudflare\.com)', log)
if match:
    hostname = match.group(1)
    print(f"\n{'='*60}")
    print(f"SSH Tunnel Ready!")
    print(f"Hostname: {hostname}")
    print(f"Password: {password}")
    print(f"\nConnect with:")
    print(f"  ssh root@{hostname}")
    print(f"{'='*60}")
else:
    print("Tunnel URL not found yet. Check with:")
    print("  !grep trycloudflare /tmp/cloudflared.log")

In [ ]:
# Cell 2: GPU Usage Logger (logs every 1 minute)
# Logs to /root/gpu_usage.log

import subprocess
import threading
import time
from datetime import datetime

LOG_FILE = "/root/gpu_usage.log"
INTERVAL = 60  # seconds

_stop_event = threading.Event()

def log_gpu_usage():
    """Log GPU usage to file every INTERVAL seconds."""
    print(f"GPU logger started. Writing to {LOG_FILE} every {INTERVAL}s")
    print("Run `tail -f /root/gpu_usage.log` in SSH to monitor.")

    while not _stop_event.is_set():
        try:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            result = subprocess.run(
                ["nvidia-smi", "--query-gpu=index,name,temperature.gpu,utilization.gpu,utilization.memory,memory.used,memory.total,power.draw",
                 "--format=csv,noheader,nounits"],
                capture_output=True, text=True, timeout=10
            )
            gpu_info = result.stdout.strip()

            log_entry = f"[{timestamp}] {gpu_info}\n"

            with open(LOG_FILE, "a") as f:
                f.write(log_entry)

            # Also print to notebook output periodically
            print(f"[{timestamp}] {gpu_info}")

        except Exception as e:
            print(f"[{datetime.now()}] Error: {e}")

        _stop_event.wait(INTERVAL)

    print("GPU logger stopped.")

# Start logger thread
_stop_event.clear()
logger_thread = threading.Thread(target=log_gpu_usage, daemon=True)
logger_thread.start()

# To stop: run _stop_event.set() in another cell